# cuDF/Java with IJava kernel https://github.com/SpencerPark/IJava

Maven Install spark-rapids-jni or use a remote repo

In [1]:
%%loadFromPOM
<dependency>
  <groupId>com.nvidia</groupId>
  <artifactId>spark-rapids-jni</artifactId>
  <version>23.10.0-SNAPSHOT</version>
  <classifier>cuda11</classifier>
</dependency>

In [2]:
import ai.rapids.cudf.*

In [3]:
TableDebug printer = TableDebug.builder().withOutput(TableDebug.Output.STDOUT).build()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


In [4]:
Table table0 = new Table.TestBuilder().column(1, 2, 3, 4).build()

In [5]:
printer.debug("table0", table0)

DEBUG table0 Table{columns=[ColumnVector{rows=4, type=INT32, nullCount=Optional[0], offHeap=(ID: 4 7fa3b8e75170)}], cudfTable=140338497710256, rows=4}
GPU COLUMN 0 - NC: 0 DATA: DeviceMemoryBufferView{address=0x7fa317200000, length=16, id=-1} VAL: null
COLUMN 0 - INT32
0 1
1 2
2 3
3 4
GPU COLUMN 0 - NC: 0 DATA: DeviceMemoryBufferView{address=0x7fa317200000, length=16, id=-1} VAL: null
COLUMN 0 - INT32
0 1
1 2
2 3
3 4


In [6]:
table0.getColumn(0).sum()

Scalar{type=INT32 value=10} (ID: 7 7fa322b989c0)